In [1]:
print("Setup: imports, helpers, reproducibility")

import numpy as np
import pandas as pd

# plotting not required; keep minimal
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api as sm
import statsmodels.formula.api as smf

# ISLP loader (install if missing)
try:
    from ISLP import load_data
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ISLP"])
    from ISLP import load_data

# Numpy print formatting for the lab's array outputs
np.set_printoptions(suppress=True)

# Scorer for MSE (positive)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)


Setup: imports, helpers, reproducibility


In [2]:
print("Q1: Validation Set — Boston (medv ~ lstat), degrees 1–4, 253/253 split, random_state=42")

# Load Boston from ISLP (this is the ISLR2 version)
Boston = load_data("Boston").copy()

# Use only lstat -> medv
X = Boston[["lstat"]].to_numpy()
y = Boston["medv"].to_numpy()

# Exactly 253 per split
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=253, random_state=42, shuffle=True
)

val_mse = []
for d in [1, 2, 3, 4]:
    model = make_pipeline(PolynomialFeatures(degree=d, include_bias=False),
                          LinearRegression())
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    val_mse.append(mse)

val_mse_arr = np.round(np.array(val_mse, dtype=float), 2)  # exactly 4 elements, 2 decimals
print("Validation MSE by degree [1,2,3,4]:")
print(val_mse_arr)   # <-- this is the deliverable for Q1


Q1: Validation Set — Boston (medv ~ lstat), degrees 1–4, 253/253 split, random_state=42
Validation MSE by degree [1,2,3,4]:
[38.51 30.84 29.22 27.75]


In [3]:
print("Q2: LOOCV — College (Outstate ~ Room.Board), degrees 1–5, cv=n (LOOCV)")

College = load_data("College").copy()

X = College[["Room.Board"]].to_numpy()
y = College["Outstate"].to_numpy()
n = len(College)

# For LOOCV: KFold with n_splits=n (leave-one-out)
kf_loocv = KFold(n_splits=n, shuffle=False)

cv_mse = []
for d in [1, 2, 3, 4, 5]:
    pipe = make_pipeline(PolynomialFeatures(degree=d, include_bias=False),
                         LinearRegression())
    # scoring returns NEGATIVE MSE; take -mean
    cvres = cross_validate(pipe, X, y, cv=kf_loocv,
                           scoring="neg_mean_squared_error",
                           return_train_score=False, n_jobs=None)
    mean_mse = -np.mean(cvres["test_score"])
    cv_mse.append(mean_mse)

cv_mse_arr = np.round(np.array(cv_mse, dtype=float), 2)
print("LOOCV MSE by degree [1,2,3,4,5]:")
print(cv_mse_arr)  # <-- deliverable for Q2


Q2: LOOCV — College (Outstate ~ Room.Board), degrees 1–5, cv=n (LOOCV)
LOOCV MSE by degree [1,2,3,4,5]:
[9291471.1  9255509.68 9263314.52 9308487.12 9391334.87]


In [4]:
print("Q3: K-Fold CV comparison — College (Outstate ~ Room.Board), degrees 1–3")
print("Difference reported as: mean_MSE_5fold - mean_MSE_10fold (rounded to 3 decimals)")

X = College[["Room.Board"]].to_numpy()
y = College["Outstate"].to_numpy()

kf5  = KFold(n_splits=5,  shuffle=True, random_state=123)
kf10 = KFold(n_splits=10, shuffle=True, random_state=123)

diffs = []
for d in [1, 2, 3]:
    pipe = make_pipeline(PolynomialFeatures(degree=d, include_bias=False),
                         LinearRegression())
    # 5-fold
    r5 = cross_validate(pipe, X, y, cv=kf5, scoring="neg_mean_squared_error",
                        return_train_score=False)
    mse5 = -np.mean(r5["test_score"])
    # 10-fold
    r10 = cross_validate(pipe, X, y, cv=kf10, scoring="neg_mean_squared_error",
                         return_train_score=False)
    mse10 = -np.mean(r10["test_score"])
    diffs.append(mse5 - mse10)

diffs_arr = np.round(np.array(diffs, dtype=float), 3)
print("Differences by degree [1,2,3] (5-fold minus 10-fold):")
print(diffs_arr)   # <-- deliverable for Q3


Q3: K-Fold CV comparison — College (Outstate ~ Room.Board), degrees 1–3
Difference reported as: mean_MSE_5fold - mean_MSE_10fold (rounded to 3 decimals)
Differences by degree [1,2,3] (5-fold minus 10-fold):
[62394.573 64506.702 91919.397]


In [5]:
print("Q4: Bootstrap SE of correlation — Default: corr(balance, income), B=2000, seed=456")

Default = load_data("Default").copy()

def corr_fn(df):
    return df["balance"].corr(df["income"])

# Actual correlation on full data
corr_actual = corr_fn(Default)

# Bootstrap
rng = np.random.default_rng(456)
n = len(Default)
B = 2000
boot_vals = np.empty(B, dtype=float)

for b in range(B):
    idx = rng.integers(0, n, size=n, endpoint=False)
    boot_vals[b] = corr_fn(Default.iloc[idx])

se_boot = np.std(boot_vals, ddof=1)

print(f"Actual corr: {corr_actual:.4f}")
print(f"Bootstrap SE: {se_boot:.4f}")   # <-- deliverables for Q4 (both rounded to 4 decimals)


Q4: Bootstrap SE of correlation — Default: corr(balance, income), B=2000, seed=456
Actual corr: -0.1522
Bootstrap SE: 0.0098


In [6]:
print("Q5: Bootstrap vs OLS SE — Wage: wage ~ age + age^2, B=1500, seed=789")
Wage = load_data("Wage").copy()

# Clean columns of interest
Wage = Wage[["wage", "age"]].dropna().copy()
Wage["age2"] = Wage["age"]**2

# OLS (quadratic)
quad = smf.ols("wage ~ age + age2", data=Wage).fit()
ols_se = quad.bse  # Intercept, age, age2

# Bootstrap SEs (resample rows)
B = 1500
rng = np.random.default_rng(789)
n = len(Wage)
coef_boot = np.empty((B, 3), dtype=float)  # [Intercept, age, age2]

for b in range(B):
    idx = rng.integers(0, n, size=n, endpoint=False)
    boot_fit = smf.ols("wage ~ age + age2", data=Wage.iloc[idx]).fit()
    coef_boot[b, :] = boot_fit.params.values  # order aligns with ols_se.index

boot_se = coef_boot.std(axis=0, ddof=1)
ratio = boot_se / ols_se.values  # same order

ratio_rounded = np.round(ratio, 3)
print("Ratios (Bootstrap SE / OLS SE) for [Intercept, age, age^2]:")
print(ratio_rounded)  # <-- deliverable for Q5


Q5: Bootstrap vs OLS SE — Wage: wage ~ age + age^2, B=1500, seed=789
Ratios (Bootstrap SE / OLS SE) for [Intercept, age, age^2]:
[0.75  0.808 0.841]
